In [ ]:
import sympy.physics.mechanics as me
import sympy as sm
from scipy.integrate import solve_ivp
import numpy as np
from itertools import permutations
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import animation
from IPython.display import HTML
import matplotlib
import time
matplotlib.rcParams['animation.embed_limit'] = 2**128
sm.init_printing(use_latex='mathjax')

This is needed to exit a loop, when a feasible location of the discs within the limitations of the walls was found.

In [ ]:
class Rausspringen(Exception):
    pass

**n discs**, named $Dmc_0....Dmc_{n-1}$ with radius $r_0$ and mass $m_0$ are sliding on the frictionless horizontal X/Z plane.\
Their space is limited by four walls: $\overline{(0, 0) (l_W, 0)}, \overline{(l_W, 0)  (l_W, l_W)}, \overline{(l_W, l_W) (0, l_W)}, \overline{(0, l_W)  (0, 0)}$.\ 
I call them $wall_0$...$wall_3$, with $wall_0$ the segment of the X - axis, and counted clock wise.\
The collision force is always on the line $\overline{Dmc_i, Dmc_j}$, $0 \le i, j \le n-1$, $i \neq j$, or on a line perpendicular to a wall through $Dmc_i$.\
There is speed dependent friction between discs, with coefficient $m_u$ and between walls and discs, with coefficient $m_{uW}$

An observer, a particle of mass $m_o$ may be attached anywhere within each disc.

**Note about the force during the collisions**

 **Hunt Crossley's method**
 
My reference is this article, given to me by JM\
https://www.sciencedirect.com/science/article/pii/S0094114X23000782 \

 
This is with dissipation during the collision, the general force is given in (63) as\
$f_n = k_0 \cdot \rho + \chi \cdot \dot \rho$, with $k_0$ as above, $\rho$ the penetration, and $\dot\rho$ the speed of the penetration.\
In the article it is stated, that $n = \frac{3}{2}$ is a good choice, it is derived in Hertz' approach. Of course, $\rho, \dot\rho$ must be the signed magnitudes of the respective vectors.

A more realistic force is given in (64) as:\
$f_n = k_0 \cdot \rho^n + \chi \cdot \rho^n\cdot \dot \rho$, as this avoids discontinuity at the moment of impact.

**Hunt and Crossley** give this value for $\chi$, see table 1:

$\chi = \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{k_0}{\dot \rho^{(-)}}$, 
where $c_\tau = \dfrac{v_1^{(+)} - v_2^{(+)}}{v_1^{(-)} - v_2^{(-)}}$, where $v_i^{(-)}, v_i^{(+)}$ are the speeds of $body_i$, before and after the collosion, see (45), $\dot\rho^{(-)}$ is the speed right at the time the impact starts. $c_\tau$ is an experimental factor, apparently around 0.8 for steel.

Using (64), this results in their expression for the force:

$f_n = k_0 \cdot \rho^n \left[1 + \dfrac{3}{2} \cdot(1 - c_\tau) \cdot \dfrac{\dot\rho}{\dot\rho^{(-)}}\right]$

with $k_0 = \frac{4}{3\cdot(\sigma_1 + \sigma_2)} \cdot \sqrt{\frac{R_1 \cdot R_2}{R_1 + R_2}}$, where $\sigma_i = \frac{1 - \nu_i^2}{E_i}$, with $\nu_i$ = Poisson's ratio, $E_i$ = Young"s modulus, $R_1, R_2$ the radii of the colliding bodies, $\rho$ the penetration depth. All is near equations (54) and (61) of this article.

As per the article, $n = \frac{3}{2}$ is always to be used.

*spring energy* =   $ k_0 \cdot \int_{0}^{\rho} k^{3/2}\,dk$ = $k_0 \cdot\frac{2}{5} \cdot \rho^{5/2}$\
I assume, the dissipated energy cannot be given in closed form, at least the article does not give one.

**Note**  
$c_\tau = 1.$ gives **Hertz's** solution to the impact problem, also described in the article.


**Variables**

- $n$: number of discs
- $q_0...q_{n-1}$: generalized coordinates for the discs
- $u_0...u_{n-1}$: the angular speeds
- $x_i, z_i$: the coordinates, in the inertial frame $N$, of the center of the i-th disc
- $ux_i, uz_i$ their speeds
- $N$: frame of inertia
- $A_i$: body fixed frame of the i-th disc

- $m_i$: mass of the i-th disc
- $Dmc_i$: center of the i-th disc
- $Po_i$: observer (particle) on i-th disc
- $\alpha_i$: distance of observer on i-th disc
- $m_W$: mass of the wall.
- $r_0, l_W, k_0$: radius of the discs, length of the wall, modulus of elasticity of the pendulum body
- $k_{0W}$: modulus of elasticity of the collision disc with a wall
- $i_{YY_i}$: moment of ineratia of the i-th disc


- $m_u$: coefficient of friction between discs / between disc and wall.
- $c_\tau$: the experimental constant needed for Hunt-Crossley
- $rhodt_{max}$: the collission speed, to be determined during integration, needed for Hunt_Crossley
- $rhodt_{wall}$: the collision speeds when $disc_i$ hits a wall

In [ ]:
start = time.time()
#==========================================
n = 4       # n > 1
#==========================================

if isinstance(n, int) == False or n < 2:
    raise Exception('n must be an integer larger than 1')
    
q_list  = me.dynamicsymbols(f'q:{n}')
u_list  = me.dynamicsymbols(f'u:{n}')
x_list  = me.dynamicsymbols(f'x:{n}')
z_list  = me.dynamicsymbols(f'z:{n}')
ux_list = me.dynamicsymbols(f'ux:{n}')
uz_list = me.dynamicsymbols(f'uz:{n}')

rhodtmax = []
for i, j in permutations(range(n), r=2):
    rhodtmax.append(sm.symbols('rhodtmax' + str(i) + str(j)))

rhodtwall  = []
for i in range(n):
    rhodtwall.append([sm.symbols('rhodtwall' + '_' + str(i) + '_' + str(j)) for j in range(4)])
        
m0, mo, r0, lW, k0, k0W, iYY, ctau, mu, muW = sm.symbols('m0, mo, r0, lW, k0, k0W, iYY, ctau, mu, muW')

N  = me.ReferenceFrame('N')
P0 = me.Point('P0')
P0.set_vel(N, 0)

t = me.dynamicsymbols._t

A_list     = sm.symbols(f'A:{n}', cls=me.ReferenceFrame)
Dmc_list   = sm.symbols(f'Dmc:{n}', cls=me.Point)
Po_list    = sm.symbols(f'Po:{n}', cls=me.Point)
alpha_list = sm.symbols(f'alpha:{n}')

Body1 = []
Body2 = []
for i in range(n):
    A_list[i].orient_axis(N, q_list[i], N.y)
    A_list[i].set_ang_vel(N, u_list[i] * N.y)
    
    Dmc_list[i].set_pos(P0, x_list[i]*N.x + z_list[i]*N.z)
    Dmc_list[i].set_vel(N, ux_list[i]*N.x + uz_list[i]*N.z)
    
    Po_list[i].set_pos(Dmc_list[i], r0 * alpha_list[i] * A_list[i].x)
    Po_list[i].v2pt_theory(Dmc_list[i], N, A_list[i])
    
    I = me.inertia(A_list[i], 0, iYY, 0)                                              
    body = me.RigidBody('body' + str(i), Dmc_list[i], A_list[i], m0, (I, Dmc_list[i]))
    teil = me.Particle('teil' + str(i), Po_list[i], mo)
    Body1.append(body)
    Body2.append(teil)
BODY = Body1 + Body2

**Colliding force between discs**

This function returns the force, as given by Hunt-Crossley, which $P_1$ excerts on $P_2$.\
The relevant speed is only the speed component in $\overline{P_1 P_2}$ direction.\
$sm.Heaviside(2. \cdot r - abstand, 0)$ ensures, that there is a force only during the collision. 

I get $\dot\rho^{(-)}$ during integration. The way I defined it, it should always be negative, just like $\dot\rho$ in the first phase of the penetration. For whatever reason, this is not always so.\
Using $\dfrac{-\dot\rho}{|\dot\rho^{(-)}|}$ gives the right results.

In [ ]:
def HC_disc(N, P1, P2, r, ctau, rhodtmax, k0):
    '''
Calculates the contact force exerted by P1 on P2, according to the Hunt-Crossley formula given above.
I assume, that the contact force always acts along the line P1/P2. I think, this is a fair assymption if
the colliding balls are homogenious.

The variables in the list are

- N is an me.ReferenceFrame, the inertial frame
- P1, P2 are me.Point objects. They are the centers of two me.RigidBody objects, here assumed to be two
  ball each of radius r
- radius of the ball
- ctau, the experimental constant needed
- rhodtmax, the relative speeds of P1 to P2, right at the impact time, measured in N. 
  This has to be calculated numerically during the integration, I found no ther way.
- k0 is the force constant


    '''
    vektorP1P2 = P2.pos_from(P1)
    abstand = vektorP1P2.magnitude() 
    richtung = vektorP1P2.normalize()        
    
    rho = (2.*r - abstand)/2.      # penetration. Positive if the two balls are in collision
    geschw = vektorP1P2.diff(t, N)
    rhodt = me.dot(geschw, richtung)
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft = k0 * rho**(3/2) * (1. + 3./2. * (1 - ctau) * (-rhodt/sm.Abs(rhodtmax))
        ) * sm.Heaviside(2. * r - abstand, 0.) * richtung
    
    return kraft   

**Collision force between disc and a wall**\
As far as the penetration depth is concerned, this is like two discs colliding: the penetration depth of the wall into the disc is the same as the disc into the wall 

In [ ]:
def HC_wall(N, P1, r, ctau, rhodtwall, k0W):
    
    '''
Calculates the contact force exerted by the four walls on P1, according to the Hunt-Crossley formula 
given above. I assume, that the contact force always acts in the direction normal to the wall. 
(Of course, a disc can maximally be in simultaneous contact with two walls)

The variables in the list are

- N is an me.ReferenceFrame, the inertial frame
- P1, is a me.Point object, the center of the disc
- radius of the disc
- ctau, the experimental constant needed
- rhodtmax, a list containing the speed right before P1 hits wall_i 
  This has to be calculated numerically during the integration, I found no ther way.
- k0W is the force constant. This is different from k0 as the wall is like a disc with R = oo.

    '''
# wall0
    abstand = me.dot(P1.pos_from(P0), N.z)     # this is simply the Z coordinate of P1
    richtung = N.z                             # points from wall0 to P1       
    
    rho = (r - abstand)       # penetration. Positive if the disc is in collision with wall0
    rhodt = me.dot(P1.vel(N), richtung)
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft0 = k0W * rho**(3/2) * (1. + 3./2. * (1 - ctau) * (-rhodt/sm.Abs(rhodtwall[0]))        
                                ) * sm.Heaviside(r - abstand, 0.) * richtung
    
# wall1
    abstand = lW - me.dot(P1.pos_from(P0), N.x) # this is the distance of P1 to wall1
    richtung = -N.x                             # points from wall1 to P1       
    
    rho = (r - abstand)       # penetration. Positive if the disc is in collision with wall0
    rhodt = me.dot(P1.vel(N), richtung)
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft1 = k0W * rho**(3/2) * (1. + 3./2. * (1 - ctau) * (-rhodt/sm.Abs(rhodtwall[1]))        
                                ) * sm.Heaviside(r - abstand, 0.) * richtung 
    
    
# wall2
    abstand = lW - me.dot(P1.pos_from(P0), N.z) # this is the distance of P1 to wall2
    richtung = -N.z                             # points from wall2 to P1       
    
    rho = (r - abstand)       # penetration. Positive if the disc is in collision with wall0
    rhodt = me.dot(P1.vel(N), richtung)
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft2 = k0W * rho**(3/2) * (1. + 3./2. * (1 - ctau) * (-rhodt/sm.Abs(rhodtwall[2]))        
                                ) * sm.Heaviside(r - abstand, 0.) * richtung 
    

# wall3
    abstand = me.dot(P1.pos_from(P0), N.x) # this is the distance of P1 to wall3
    richtung = N.x                             # points from wall3 to P1       
    
    rho = (r - abstand)       # penetration. Positive if the disc is in collision with wall0
    rhodt = me.dot(P1.vel(N), richtung)
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    
    kraft3 = k0W * rho**(3/2) * (1. + 3./2. * (1 - ctau) * (-rhodt/sm.Abs(rhodtwall[3]))
        ) * sm.Heaviside(r - abstand, 0.) * richtung 
    
    return kraft0 + kraft1 + kraft2 + kraft3   

*Friction when a disc hits another disc*\

This website:
https://math.stackexchange.com/questions/2195047/solve-the-vector-cross-product-equation


gives: $b = a \times x \rightarrow x = \dfrac{b \times a}{|a|^2}$

This way, I can easily get the force acting on P1, without any further geometric considerations.

In [ ]:
def friction_disc(N, A1, A2, P1, P2, r, ctau, rhodtmax, k0, mu):
    '''
when two discs collide, their surface speed in general will be different.
There is a force caused by friction. Here I calculate the force acting on the contact point CP1 of P1 in
direction of the tangent on disc1, whose center is P1, proportional to
* v(CP2) - v(CP1)
* the magnitude of the collision force which P2 excerts on P1
* the coefficient of friction mu
The force on CP is equivalent to a force on Dmc and a torque on A
    '''
    CP1, CP2 = sm.symbols('CP1, CP2', cls=me.Point)
    
    abstand = P2.pos_from(P1)
    CP1.set_pos(P1, abstand / 2.)
    CP1.v2pt_theory(P1, N, A1)
    
    CP2.set_pos(P2, -abstand / 2.)
    CP2.v2pt_theory(P2, N, A2)
    delta_v = CP2.vel(N) - CP1.vel(N)

    force_coll = HC_disc(N, P2, P1, r, ctau, rhodtmax, k0).magnitude()
    force_fric = (force_coll * mu * delta_v)

    torque = 0.5 * abstand.cross(force_fric) * sm.Heaviside(2. * r - abstand.magnitude())
    
    kraft  = 1./(0.25 * me.dot(abstand, abstand)) * torque.cross(abstand) * sm.Heaviside( 2. * 
            r - abstand.magnitude())

#    print(me.find_dynamicsymbols(kraft, reference_frame=N))
    return [kraft, torque]

*Friction when a disc hits a wall*

In [ ]:
def friction_wall(N, A1, P1, r, ctau, rhodtwall, k0W, mu):
    '''
When a disc collides with a wall, there will be a force due to friction.
I calculate the force due to friction on the contact point CP1.
This force is parallel to the wall, and proportional to
* the speed of CP1
* the magnitude of the collision force
* the coefficient of friction mu
Of course, a disc can never hit more than two wall simultaneously.
    '''
    
# wall 0
    abstand = me.dot(P1.pos_from(P0), N.z) * N.z
    
    CP0 = me.Point('CP0')
    CP0.set_pos(P1, -abstand)
    CP0.v2pt_theory(P1, N, A1)
    force_coll = HC_wall(N, P1, r, ctau, rhodtwall, k0W).magnitude()
    force_fric = -force_coll * mu * CP0.vel(N)
    
    torque0 = CP0.pos_from(P1).cross(force_fric) * sm.Heaviside(r - abstand.magnitude())
    kraft0  = 1./me.dot(abstand, abstand) * torque0.cross(-abstand) * sm.Heaviside(r - abstand.magnitude())

# wall 1
    abstand = lW*N.x - me.dot(P1.pos_from(P0), N.x) * N.x
    
    CP1 = me.Point('CP1')
    CP1.set_pos(P1, abstand)
    CP1.v2pt_theory(P1, N, A1)

    force_coll = HC_wall(N, P1, r, ctau, rhodtwall, k0W).magnitude()
    force_fric = -force_coll * mu * CP1.vel(N)
    
    torque1 = CP1.pos_from(P1).cross(force_fric) * sm.Heaviside(r - abstand.magnitude())
    kraft1  = 1./me.dot(abstand, abstand) * torque1.cross(abstand) * sm.Heaviside(r - abstand.magnitude())

# wall 2
    abstand = lW*N.z - me.dot(P1.pos_from(P0), N.z) * N.z
    CP2 = me.Point('CP2')
    CP2.set_pos(P1, abstand)
    CP2.v2pt_theory(P1, N, A1)

    force_coll = HC_wall(N, P1, r, ctau, rhodtwall, k0W).magnitude()
    force_fric = -force_coll * mu * CP2.vel(N)
    
    torque2 = CP2.pos_from(P1).cross(force_fric) * sm.Heaviside(r - abstand.magnitude())
    kraft2  = 1./me.dot(abstand, abstand) * torque2.cross(abstand) * sm.Heaviside(r - abstand.magnitude())

    
# wall 3
    abstand = me.dot(P1.pos_from(P0), N.x) * N.x

    CP3 = me.Point('CP3')
    CP3.set_pos(P1, -abstand)
    CP3.v2pt_theory(P1, N, A1)

    force_coll = HC_wall(N, P1, r, ctau, rhodtwall, k0W).magnitude()
    force_fric = -force_coll * mu * CP3.vel(N)
    
    torque3 = CP3.pos_from(P1).cross(force_fric) * sm.Heaviside(r - abstand.magnitude())
    kraft3  = 1./me.dot(abstand, abstand) * torque3.cross(-abstand) * sm.Heaviside(r - abstand.magnitude())
    
    return [kraft0 + kraft1 + kraft2 + kraft3, torque0 + torque1 + torque2 + torque3]   

**force from the collision of discs with discs and discs with walls**

With *permutations(range(n), r=2)* I get all possible forces of $disc_j$ on $disc_i$, $0 \le i, j \le n-1$, $i \neq j$\
This is not the most efficient way to do it, as for example $F^{disc_i \space \backslash \space disc_j} = -F^{disc_j \space \backslash \space disc_i}$, but this way seems easier to keep it all 'straight'.

In [ ]:
FL = []

#------------------------------------------------------------------------------------------------------
# Hunt - Crossley type collision forces between discs and between disc and wall
zaehler = -1
for i, j in permutations(range(n), r=2):
    zaehler += 1
    FL.append((Dmc_list[i], HC_disc(N, Dmc_list[j], Dmc_list[i], r0, ctau, rhodtmax[zaehler], k0)))

for i in range(n):
    FL.append((Dmc_list[i], HC_wall(N, Dmc_list[i], r0, ctau, rhodtwall[i], k0W )))

#------------------------------------------------------------------------------------------------------  
# Friction forces between discs
zaehler = 0
for i, j in permutations(range(n), r=2):
    zaehler += 1
    FL.append((Dmc_list[i], friction_disc(N, A_list[i], A_list[j], Dmc_list[i], Dmc_list[j], r0, ctau,
        rhodtmax[i], k0, mu)[0]))
    FL.append((A_list[i], friction_disc(N, A_list[i], A_list[j], Dmc_list[i], Dmc_list[j], r0, ctau,
        rhodtmax[i], k0, mu)[1]))
    
#------------------------------------------------------------------------------------------------------
# Friction forces between disc and wall
for i in range(n):
    FL.append((Dmc_list[i], friction_wall(N, A_list[i], Dmc_list[i], r0, ctau, rhodtwall[i],
            k0W, muW)[0]))
    FL.append((A_list[i], friction_wall(N, A_list[i], Dmc_list[i], r0, ctau, rhodtwall[i],
            k0W, muW)[1]))

**Kane's equations** 

In [ ]:
kd = [i - sm.Derivative(j, t) for i, j in zip(u_list + ux_list + uz_list, q_list + x_list + z_list)]

q_ind = x_list + z_list + q_list
u_ind = ux_list + uz_list + u_list

KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
print('MM DS', me.find_dynamicsymbols(MM))
print('MM free symbols', MM.free_symbols)
print('MM contains {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

force = KM.forcing_full
print('force DS', me.find_dynamicsymbols(force), '\n')
print('force free symbols', force.free_symbols, '\n')
print('force contains {} operations'.format(sum([force[i].count_ops(visual=False) for i in range(force.shape[0])])), '\n')


Here various **functions** are defined, which are needed later.

- *rhomax_list*: It is used during integration to calculate the speeds just before impact between $disc_j$ and $disc_i$, $0 \le i, j \le n-1$, $i \neq j$
- *rhowall_list*: It is used during integration to calculate the speeds just before impact between $disc_i$ and the walls.
- *Dmc_pos*: Holds the locations of the centers of the discs. Only for plotting.
- *Po_pos*: Holds the locations of each pbserver. Dto.
- *Dmc_distanz*: Holds the distance between $disc_j$ and $disc_i$, $0 \le i, j \le n-1$, $i \neq j$. Needed during integration
- *kinetic_energie*: calculates the kinetic energy of the bodies and particles.
- *spring_energie*: calculates the spring energy of the colliding bodies.

In [ ]:
derivative_dict = {sm.Derivative(i, t): j for i, j in zip(x_list + z_list, ux_list + uz_list)}

rhomax_list = []
for i, j in permutations(range(n), r=2):
    vektor = Dmc_list[i].pos_from(Dmc_list[j])
    richtung = vektor.normalize()
    geschw = vektor.diff(t, N).subs(derivative_dict)
    rhodt = me.dot(geschw, richtung)
    rhomax_list.append(rhodt)
print('rhomax_list DS:', set.union(*[me.find_dynamicsymbols(rhomax_list[k]) 
        for k in range(len(rhomax_list))]), '\n')
print('rhomax_list free symbols:', set.union(*[rhomax_list[k].free_symbols 
        for k in range(len(rhomax_list))]), '\n')


rhowall_list = []
for i in range(n):
    hilfs = []
    for richtung in (N.z, -N.x, -N.z, N.x):
        rhodt = me.dot(Dmc_list[i].vel(N), richtung)
        hilfs.append(rhodt)
    rhowall_list.append(hilfs)
print('rhowall_list DS:', set.union(*[me.find_dynamicsymbols(rhowall_list[k][l]) 
        for k in range(len(rhowall_list)) for l in range(4)]), '\n')
print('rhowall_list free symbols:', set.union(*[rhowall_list[k][l].free_symbols 
        for k in range(len(rhowall_list)) for l in range(4)]))

Po_pos  = [[me.dot(Po_list[i].pos_from(P0), uv) for uv in (N.x, N.z)] for i in range(n)]

Dmc_distanz = [Dmc_list[i].pos_from(Dmc_list[j]).magnitude() for i, j in permutations(range(n), r=2)]

kin_energie = sum([body.kinetic_energy(N) for body in BODY])

spring_energie = 0.
# 1. collisions of discs
for i in range(n-1):
    for j in range(i+1, n):
        distanz = Dmc_list[i].pos_from(Dmc_list[j]).magnitude()
        rho = sm.Max((2.*r0 - distanz)/2., sm.S(0.))
        rho = rho**(5/2)
        spring_energie += 2. * k0 * 2./5. * rho * sm.Heaviside(2.*r0 - distanz, 0.)
# 2. Collision of discs with walls   
for i in range(n):
# wall0
    abstand = sm.Abs(me.dot(Dmc_list[i].pos_from(P0), N.z))     # this is simply the Z coordinate of P1
    rho = (r0 - abstand)       # penetration. Positive if the disc is in collision with wall0
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    rho = rho**(5/2)
    spring_energie += k0W * 2./5. * rho * sm.Heaviside(r0 - abstand, 0.)
# wall1
    abstand = lW - sm.Abs(me.dot(Dmc_list[i].pos_from(P0), N.x))     # this is simply the Y coordinate of P1
    rho = (r0 - abstand)       # penetration. Positive if the disc is in collision with wall0
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    rho = rho**(5/2)
    spring_energie += k0W * 2./5. * rho * sm.Heaviside(r0 - abstand, 0.)
# wall2
    abstand = lW - sm.Abs(me.dot(Dmc_list[i].pos_from(P0), N.z))     # this is simply the Y coordinate of P1
    rho = (r0 - abstand)       # penetration. Positive if the disc is in collision with wall0
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    rho = rho**(5/2)
    spring_energie += k0W * 2./5. * rho * sm.Heaviside(r0 - abstand, 0.)
# wall3
    abstand = sm.Abs(me.dot(Dmc_list[i].pos_from(P0), N.x))     # this is simply the Y coordinate of P1
    rho = (r0 - abstand)       # penetration. Positive if the disc is in collision with wall0
    rho = sm.Max(rho, sm.S(0))   # if rho < 0., rho**(3/2) will give problems
    rho = rho**(5/2)
    spring_energie += k0W * 2./5. * rho * sm.Heaviside(r0 - abstand, 0.)

*Lambdification*\
The sympy functions are converted to numpy functions for numerical calculations.

In [ ]:
qL  = q_ind + u_ind
pL  = [m0, mo, r0, lW, k0, k0W, iYY, ctau, mu, muW] + list(alpha_list) + rhodtmax + rhodtwall
pL1 = [m0, mo, r0, lW, k0, k0W, iYY, ctau, mu, muW]
pL2 = [m0, mo, r0, lW, k0, k0W, iYY, ctau, mu, muW] + list(alpha_list)

MM_lam    = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True) 

rhomax_list_lam  = sm.lambdify(qL + pL1, rhomax_list, cse=True)
rhowall_list_lam = sm.lambdify(qL + pL1, rhowall_list, cse=True)
Dmc_distanz_lam  = sm.lambdify(x_list + z_list, Dmc_distanz, cse=True)

#Dmc_pos_lam = sm.lambdify(qL + pL2, Dmc_pos, cse=True)
Po_pos_lam  = sm.lambdify(qL + pL2, Po_pos, cse=True)

kin_lam    = sm.lambdify(qL + pL2, kin_energie, cse=True)
spring_lam = sm.lambdify(qL + pL2, spring_energie, cse=True)

print(f'it took {time.time()-start:.3f} sec to establish Kanes equation for {n} bodies')

**Set initial conditions and parameters**

1.\
The discs are randomly placed within the walls, such that they have a distance of at least $r_0$ from the walls, and they have a distance of at least $r_0$ from one another. If this cannot be found after 200 trials an exception is raised. As soon as a good placement is found, the loop is left\
2.\
Assign random linear speeds to each disc, in the range [-5., 5.] for each component.\
3.\
Assign arbitray non zero values to rhodtmax and rhowall. They will be overwritten during the integration and (hopefully) filled with the correct values.\
4.\
Calculate $k_0$ and $k_{0W}$. I model the wall as a disc with $r_W \to \infty$, all discs have same radius $r_0$\
Now $\displaystyle \lim_{r_W \to \infty} \sqrt{\frac{r_0 \cdot r_W}{r_0 + r_W}} = \sqrt{r_0}$, and $\sqrt{\frac{r_0 \cdot r_0}{r_0 + r_0}} = \sqrt{\frac{r_0}{2}}$\
I make the assumption, that the material constants of discs and walls are the same.\
If Young's modulus is large, the integration runs 'forever'. I use $2\cdot 10^3$.

In [ ]:
start1 = time.time()

# Input variables
#-------------------------------------------------------------------
r01   = 1.0           # radius of the pendulum
lW1   = 11.            # length of each wall
m01   = 1.            # mass of a pendulum
mo1   = 0.1           # mass of the observer

mu1   = 0.1           # friction between discs
muW1  = 0.1           # friction between disc and wall
ctau1 = 0.9          # given in the article

# initial conditions for the rotation of each disc
q_list1 = [0. for _ in range(len(q_list))]

# location of the observers
alpha_list1 = [0.8 for _ in range(len(alpha_list))]
#-------------------------------------------------------------------
#np.random.seed(11)
# 1. randomly place the discs as described above
zaehler = 0
while zaehler <= 200:
    zaehler += 1
    try:
        x_list1 = np.random.choice(np.linspace(2.*r01, lW1 - 2.*r01, 100), size=n, replace=False)
        z_list1 = np.random.choice(np.linspace(2.*r01, lW1 - 2.*r01, 100), size=n, replace=False)
        test = np.all(np.array(Dmc_distanz_lam(*x_list1, *z_list1)) - 3.*r01 > 0.)
        x_list1 = list(x_list1)
        z_list1 = list(z_list1)
        
        if test == True:
            raise Rausspringen
    except:
        break

if zaehler <= 200:
    print(f'it took {zaehler} rounds to get valid initial conditions')
    print('distance between discs is', [f'{np.array(Dmc_distanz_lam(*x_list1, *z_list1))[l] - 2.*r01:.2f}' 
                for l in range(len(Dmc_distanz))])
else:
    raise Exception(' no good location for discs found, make lW0 larger, or try again.')

# 2. Assign random linear and angular speeds to each disc
ux_list1 = np.random.choice(np.linspace(-5., 5., 100), size=n)
uz_list1 = np.random.choice(np.linspace(-5., 5., 100), size=n)
u_list1  = np.random.choice(np.linspace(-5., 5., 100), size=len(q_list))

ux_list1 = list(ux_list1)
uz_list1 = list(uz_list1)
u_list1  = list(u_list1)

# 3. Assign non-zero values to rhodtmax and rhodtwall
rhodtmax1  = [1. + k for k in range(len(rhodtmax))]
rhodtwall1 = [[1+l+k for l in range(4)] for k in range(n)]

# 4. Calculate k01 and k0W1
nu = 0.28                       # Poisson's ratio for steel, from the internet
EY = 2.e3                       # units: N/m^2, Young's modulus from the internet, around 2*10^11 for steel
sigma = (1 - nu**2) / EY
k01  = 4. / (3.* (sigma + sigma)) * np.sqrt(r01/2.)
k0W1 = 4. / (3.* (sigma + sigma)) * np.sqrt(r01)

iYY1 = 0.25 * m01 * r01**2      # from the internet

y0       = x_list1 + z_list1 + q_list1 + ux_list1 + uz_list1 + u_list1 
pL_vals  = [m01, mo1, r01, lW1, k01, k0W1, iYY1, ctau1, mu1, muW1] + alpha_list1 + rhodtmax1 + rhodtwall1
pL1_vals = [m01, mo1, r01, lW1, k01, k0W1, iYY1, ctau1, mu1, muW1]
pL2_vals = [m01, mo1, r01, lW1, k01, k0W1, iYY1, ctau1, mu1, muW1] + list(alpha_list1)
print('initial conditions are:', [f'{pL_vals[l]}' for l in range(len(pL_vals))])

**Numerical integration**

$\dot \rho^{(-)}$ has to be found numerically during integration, at least I could not think of any other way.\
If $\dot \rho^{(-)} \approx 0.$ I set it to $10^{-12}$ to avoid numerical issues. \
I use *max_step* in solve_ivp to ensure it does not miss the time close to a collision.\
I set *schritte* a bit larger than the number of function calls of solve_ivp, to ensure I get the collision times.

In [ ]:
start1 = time.time()

intervall = 10.                          
max_step = 0.001
schritte = 20000     
times = np.linspace(0, intervall, schritte)

def gradient(t, y, args):
    
# here I find rhodtmax, the collision speed of two discs just before the impact
    zaehler = -1
    for i, j in permutations(range(n), r=2):
        zaehler += 1
        if 0. < Dmc_distanz_lam(*[y[l] for l in range(2*n)])[zaehler] - 2.*args[2] < 2. * max_step:
            hilfs1 = rhomax_list_lam(*y, *pL1_vals)[zaehler]
            if np.abs(hilfs1) < 1.e-12:
                hilfs1 = np.sign(hilfs1) * 1.e-12
            args[10 + n + zaehler] = hilfs1

# Here I find rhodtwall, the collision speed between a disc and a wall
    for i in range(n):
# wall 0
        laenge = len(rhodtmax)
        abstand = y[i+n]
        if 0. < args[2] - abstand < 2.*max_step:
            hilfs1 = rhowall_list_lam(*y, *pL1_vals)[i][0]
            if np.abs(hilfs1) < 1.e-12:
                hilfs1 = np.sign(hilfs1) * 1.e-12
            args[10 + n + laenge + i][0] = hilfs1
            
# wall 1
        laenge = len(rhodtmax)
        abstand = args[3] - y[i]
        hilfs1 = rhowall_list_lam(*y, *pL1_vals)[i][1]
        if np.abs(hilfs1) < 1.e-12:
            hilfs1 = np.sign(hilfs1) * 1.e-12
        if 0. < args[2] - abstand < 2.*max_step:
            args[10 + n + laenge + i][1] = hilfs1
            
# wall 2
        laenge = len(rhodtmax)
        abstand = args[3] - y[n+i]
        if 0. < args[2] - abstand < 2.*max_step:
            hilfs1 = rhowall_list_lam(*y, *pL1_vals)[i][2]
            if np.abs(hilfs1) < 1.e-12:
                hilfs1 = np.sign(hilfs1) * 1.e-12
            args[10 + n + laenge + i][2] = hilfs1
            
# wall 3
        laenge = len(rhodtmax)
        abstand = y[i]
        if 0. < args[2] - abstand < 2.*max_step:
            hilfs1 = rhowall_list_lam(*y, *pL1_vals)[i][3]
            if np.abs(hilfs1) < 1.e-12:
                hilfs1 = np.sign(hilfs1) * 1.e-12
            args[10 + n + laenge + i][3] = hilfs1
            
        
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]

    
resultat1 = solve_ivp(gradient, (0., float(intervall)), y0, args=(pL_vals,), t_eval=times, method='BDF' 
            , max_step=max_step, atol=1.e-6, rtol=1.e-6)
resultat = resultat1.y.T
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status])

print('to calculate an intervall of {:.0f} sec it took {} loops and {:.3f} sec running time'.
      format(intervall, resultat1.nfev, time.time() - start1))


*plot* whichever generalized coordinates you want to see.\
*schritte* is very large, to catch the impacts, this is not needed her. Hence I reduce the number of points to be plotted to around $N_2$.

In [ ]:
N2 = 500
N1 = int(resultat.shape[0] / N2)
times1 = []
resultat1 = []
for i in range(resultat.shape[0]):
    if i % N1 == 0:
        times1.append(times[i])
        resultat1.append(resultat[i])
resultat1 = np.array(resultat1)
times1 = np.array(times1)

bezeichnung = (['x' + str(i) for i in range(n)] +
               ['z' + str(i) for i in range(n)] +
               ['q' + str(i) for i in range(n)] + 
               ['ux' + str(i) for i in range(n)] +
               ['uz' + str(i) for i in range(n)] +
               ['u' + str(i) for i in range(n)]  
              
              ) 
fig, ax = plt.subplots(figsize=(10,5))
for i in range(5*n, 6*n):
    label = 'gen. coord. ' + str(i)
    ax.plot(times1, resultat1[:, i], label=bezeichnung[i])
ax.set_title('Generalized coordinates')
ax.set_xlabel('time (sec)')
ax.legend();

Plot the **hysteresis curve** of $disc_i$ when it collides with $wall_0$ and $disc_i$ when it collides with $disc_j$, $i < j$\
Hysteresis curves of subsequent collisions are nested, as the energy dissipated is less on every subsequent collision.\
The $i_0$ is needed to get the (approximately) correct $\dot \rho^{(-)}$, the speed at the impact.\
I only plot, if an impact did take place.\
The black numbers on the curves give the time of the impact.

In [ ]:
for l1 in range(n):
    HC_kraft = []
    HC_displ = []
    HC_times = []
    zaehler  = 0

    i0 = 0
    for i in range(resultat.shape[0]):
        abstand = r01 - resultat[i, n+l1]
        if abstand < 0.:
            i0 = i+1

        if abstand >= 0. and i0 == i:
            walldt = resultat[i, 4*n+l1]
        if abstand >= 0.:
            rhodt = resultat[i, 4*n+l1]
            kraft0 = k0W1 * abstand**(3/2) * (1. + 3./2. * (1 - ctau1) * rhodt/walldt)
            HC_displ.append(abstand)
            HC_kraft.append(kraft0)
            HC_times.append((zaehler, times[i]))
            zaehler +=1
    if len(HC_displ) > 0:
        HC_displ = np.array(HC_displ)
        HC_kraft = np.array(HC_kraft)

        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(HC_displ, HC_kraft)
        ax.set_xlabel('penetration depth (m)')
        ax.set_ylabel('contact force (N)')
        ax.set_title(f'hysteresis curves of successive impacts of dics_{l1} with wall_0, ctau = {ctau1}' + 
                     f'mu = {mu1}, muW = {muW1}');
    
        zeitpunkte = 8
        reduction = max(1, int(len(HC_times)/zeitpunkte))
        for k in range(len(HC_times)):
            if k % reduction == 0:
                coord  = HC_times[k][0]
                ax.text(HC_displ[coord], HC_kraft[coord], f'{HC_times[k][1]:.2f}', color="black")


# plot the hysteresis curve of disc i colliding with disc j, for i < j only,
zaehler1 = -1
for l1, l2 in permutations(range(n), r=2):
    zaehler1 += 1
    HC_kraft = []
    HC_displ = []
    HC_times = []
    zaehler  = 0

    i0 = 0
    for i in range(resultat.shape[0]):
        abstand = 2.*r01 - Dmc_distanz_lam(*[resultat[i, j] for j in range(2*n)])[zaehler1]
        if abstand < 0.:
            i0 = i+1

        if abstand >= 0. and i0 == i:
            walldt = rhomax_list_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
                    *pL1_vals)[zaehler1]
        if abstand >= 0.:
            rhodt  = rhomax_list_lam(*[resultat[i, j] for j in range(resultat.shape[1])], 
                    *pL1_vals)[zaehler1]
            kraft0 = k01 * abstand**(3/2) * (1. + 3./2. * (1 - ctau1) * rhodt/walldt)
            HC_displ.append(abstand)
            HC_kraft.append(kraft0)
            HC_times.append((zaehler, times[i]))
            zaehler +=1
    
    if len(HC_displ) > 0 and l1 < l2:
        HC_displ = np.array(HC_displ)
        HC_kraft = np.array(HC_kraft)

        fig, ax = plt.subplots(figsize=(10,5))
        ax.plot(HC_displ, HC_kraft, color='red')
        ax.set_xlabel('penetration depth (m)')
        ax.set_ylabel('contact force (N)')
        ax.set_title(f'hysteresis curves of successive impacts of dics_{l1} with disc_{l2}' +
                     f' ctau = {ctau1}, mu = {mu1}, muW = {muW1}');
    
        zeitpunkte = 12
        reduction = max(1, int(len(HC_times)/zeitpunkte))
        for k in range(len(HC_times)):
            if k % reduction == 0:
                coord  = HC_times[k][0]
                ax.text(HC_displ[coord], HC_kraft[coord], f'{HC_times[k][1]:.2f}', color="black")
    

Plot the **energies** of the system.

If $c_{tau} < 1.$, or $m_u \neq 0.$ or $m_{uW} \neq 0.$ the total energy should drop monotonically. This is due to the Hunt-Crossley prescription of the forces during a collision, due to friction during the collisions respectively.

In [ ]:
kin_np    = np.empty(schritte)
spring_np = np.empty(schritte)
total_np  = np.empty(schritte)


for i in range(schritte):
    kin_np[i]    = kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL2_vals)
    spring_np[i] = spring_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL2_vals)
    total_np[i]  = spring_np[i] + kin_np[i]

fig, ax = plt.subplots(figsize=(10, 5))
for i, j in zip((kin_np, spring_np, total_np), ('kinetic energy', 'spring energy', 'total energy')):
    ax.plot(times, i, label=j)
ax.set_xlabel('time (sec)')
ax.set_ylabel('Energy (Nm)')
ax.set_title(f'Energies of the system with {n} balls, with ctau = {ctau1}, mu = {mu1} and muW = {muW1}')
ax.legend();

*Animation*

As the number of points in time, given as *schritte* is verly large, I limit to around *zeitpunkte*. Otherwise it would take a very long time to finish the animation.\
The location of the walls is reflected and rotated relative to my definition given above. This is a result of the orientation of the axis in matplotlib. As it does not really matter, I did not bother to 'correct' it.\
The size of the discs, and the depth of penetration is not to scale. I would not know, how to do this.

In [ ]:
times2 = []
resultat2 = []

#=======================
zeitpunkte = 500
#=======================

reduction = max(1, int(len(times)/zeitpunkte))

for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2.append(resultat[i])

schritte2 = len(times2)
resultat2 = np.array(resultat2)
times2 = np.array(times2)
print('number of points considered:',len(times2))

Dmc_X = np.array([[resultat2[i, j] for j in range(n)] for i in range(schritte2)])
Dmc_Z = np.array([[resultat2[i, j] for j in range(n, 2*n)] for i in range(schritte2)])

Po_X = np.empty((schritte2, n))
Po_Z = np.empty((schritte2, n))
for i in range(schritte2):
    Po_X[i] = [Po_pos_lam(*[resultat2[i, j] for j in range(resultat.shape[1])], *pL2_vals)[l][0] 
            for l in range(n) ]
    Po_Z[i] = [Po_pos_lam(*[resultat2[i, j] for j in range(resultat.shape[1])], *pL2_vals)[l][1] 
            for l in range(n)]

farben = ['red', 'green', 'blue', 'orange', 'yellow', 'pink']
if n > 6:
    raise Exception('define more colors in the list farben')
    
def animate_pendulum(times2, Dmc_x, Dmc_Z, Po_X, Po_Z):

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.axis('off')
    ax.set(xlim=(- 1., lW1 + 1.), ylim=( - 1., lW1 + 1.))
    ax.plot([0, lW1], [0, 0], 'bo', linewidth=2, linestyle='-', markersize=0)
    ax.text(lW1/2., -0.5, 'wall 0')
    ax.plot([lW1, lW1], [0, lW1], 'bo', linewidth=2, linestyle='-', markersize=0)
    ax.text(lW1 + 0.1, lW1/2.+0.1, 'wall 1')
    ax.plot([lW1, 0], [lW1, lW1], 'bo', linewidth=2, linestyle='-', markersize=0)
    ax.text(lW1/2. + 0.1, lW1 + 0.1, 'wall 2')
    ax.plot([0, 0], [lW1, 0], 'bo', linewidth=2, linestyle='-', markersize=0)
    ax.text(-1., lW1/2., 'wall 3')
    
    
    LINE1 = []
    LINE2 = []
    LINE3 = []
    for i in range(n):
# picking the 'right' radius of the discs I do by trial and error. I did not try to get a formula
        line1, = ax.plot([], [], 'o', markersize=65*11./lW1)
        line2, = ax.plot([], [], 'o', markersize=5, color='black')
        line3, = ax.plot([], [], '-', markersize=0, linewidth=0.3)
        LINE1.append(line1)
        LINE2.append(line2)
        LINE3.append(line3)

    def animate(i):
        ax.set_title(f'System with {n} bodies, running time {i/schritte2 * intervall:.2f} sec' + '\n' +
                     f' ctau = {ctau1}, mu = {mu1}, muW = {muW1}', fontsize=12)
        for j in range(n):
            LINE1[j].set_data(Dmc_X[i, j], Dmc_Z[i, j])
            LINE1[j].set_color(farben[j])
            LINE2[j].set_data(Po_X[i, j], Po_Z[i, j])
            LINE3[j].set_data(Dmc_X[:i, j], Dmc_Z[:i, j])
            LINE3[j].set_color(farben[j])
        return LINE1 + LINE2 + LINE3

    anim = animation.FuncAnimation(fig, animate, frames=schritte2,
                                   interval=1000*times2.max() / schritte2,
                                   blit=True)
    plt.close(fig)
    return anim

anim = animate_pendulum(times2, Dmc_X, Dmc_Z, Po_X, Po_Z)
print(f'it took {time.time() - start :.3f} sec to run the program, BEFORE HTML')
HTML(anim.to_jshtml())